# Vanilla Neural Network
I think this question is less convenient to tackle using vanilla neural network than using sequential models like RNN, LSTM, etc. This is because vanilla NN normally deals with input with **fixed length**. To accomodate our need of different length from `2` to `10`, we would have to train `9` models, each with input shape `2, 3, 4, ...`.

**Note**. Creating `10-2+1 = 9` NNs is not a bad thing. Their cooperated performance might also not mediocre. It is just that I find it more challenging/rewarding to try to find a devise-once-use-everywhere solution, so I ended up spending most of the time on finding such solutions. 

## Correction
Actually, there exist ways to still use vanilla NN for this question. One such way is through padding. More precisely, let's take a few examples to illustrate our point:

01. Input array `[0, 9, 7, 1, 2]`
  - We may use integers $\in {91, 92, 93, \ldots, 98}$ to pad, whence the padded input array becomes
  `[0, 9, 7, 1, 2, 91, 92, 93, 94, 95]`. We always pad until the padded array has length `10`.
  - As for the corresponding output, I choose to return the permutation which make the padded input array sorted.
  In this particular example,  that would be `[0, 3, 4, 2, 1, 5, 6, 7, 8, 9]`. Note how the last five indices
  have not been altered at all in this permutation.
02. Input array `[9, 0]`
  - The padded input array would be `[9, 0, 91, 92, 93, 94, 95, 96, 97, 98]`.
  - The output would be `[1, 0, 2, 3, 4, 5, 6, 7, 8, 9]`.

This looks a little involved and artificial; nevertheless, it also brings convenience

- The input and output are now both of fixed shape
- The fact of being of fixed shape makes creating the dataset a lot more easier, (which in turn makes splitting it into Train/Validation/Test sets easier).

### Correction inside correction
We have said that we wanted to use `91..98` as padders, but, sorry, because normally we would use one-hot encoding during the dataset preparation for `X`. It's clueless how these `91..98` should be mapped. So, let's just forget the `91..98` idea, just **pad with zero vectors**.

In [1]:
padders = list(range(91, 98+1))
padders

[91, 92, 93, 94, 95, 96, 97, 98]

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from functools import reduce
from itertools import combinations, permutations
from math import factorial

The following `X` will be our dataset (including training/validation/test sets).

In [3]:
n_classes = 10
max_length = 10
n_instances = sum([reduce(lambda x, y: x*y, range(n_classes,n_classes-length,-1)) for length in range(2, max_length+1)])
n_instances

9864090

In [4]:
X = np.zeros((n_instances, max_length, n_classes), dtype=np.float32)

I have said in `README.md` that CNN is of little use here because we are not dealing with images. However, the shape of `X` does look like a single-channel image. Still, using CNN to extract local features makes little sense, so we will probably stick to our plan -- Maybe the first layer of our vanilla NN would be a `keras.layers.Flatten` and followed by a few fully connected layers.

In [5]:
np.argsort([9,5,0,3])

array([2, 3, 1, 0])

In [6]:
A = np.array([9,5,0,3])
A[np.argsort(A)]

array([0, 3, 5, 9])

In [7]:
def one_hot(array, depth=n_classes):
    """
    array is an ndarray of shape (None,)
    """
    return np.eye(depth)[array, :]

In [8]:
one_hot(A)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

In [9]:
tf.one_hot(A, depth=10)

<tf.Tensor: shape=(4, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [10]:
# labels
Y = np.empty((n_instances, max_length), dtype=np.float32)  

In [11]:
np.concatenate(([1,2,3], [4,5,6]))

array([1, 2, 3, 4, 5, 6])

In [12]:
p = (4,3,9,7,8)
length = len(p)
np.concatenate((np.argsort(p), np.arange(length, max_length)))

array([1, 0, 3, 4, 2, 5, 6, 7, 8, 9])

In [13]:
%%time
X[...] = 0
S = set(range(0, 9+1))
index_instance = 0
#for length in tqdm(range(2, max_length+1)):
for length in range(2, max_length+1):    
    n_permutations = factorial(length)
    #n_combinations = n_instances // n_permutations
    #for i, c in enumerate(combinations(S, length)):
    for c in combinations(S, length):
        #for j, p in enumerate(permutations(c)):
        for p in permutations(c):
            #print(f"(index_instance/n_instances = {index_instance}/{n_instances})", end="\r")
            #print(f"np.array(p) = {np.array(p)}")
            X[index_instance, :length, :] = one_hot(np.array(p))#[..., np.newaxis]
            Y[index_instance, :] = np.concatenate((np.argsort(p), np.arange(length, max_length)))
            #print(f"""
            #(index_instance/n_instances = {index_instance}/{n_instances})
            #x = {one_hot(np.array(p))}
            #y = {np.concatenate((np.argsort(p), np.arange(length, max_length)))}
            #""", end="\r")
            index_instance += 1

CPU times: user 2min 27s, sys: 1.17 s, total: 2min 29s
Wall time: 2min 29s


**(?)** Improvement. The above construction of `X, Y` takes a little long (6min on Thinkpad X200), can consider using concurrent programming.

In [14]:
index = half = n_instances // 2
print(f"X[index] =\n{X[index]}")
print(f"Y[index] = {Y[index]}")

X[index] =
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Y[index] = [5. 8. 2. 0. 6. 4. 1. 7. 3. 9.]


In [15]:
index = -1
print(f"X[index] =\n{X[index]}")
print(f"Y[index] = {Y[index]}")

X[index] =
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Y[index] = [9. 8. 7. 6. 5. 4. 3. 2. 1. 0.]


In [16]:
index = 2
print(f"X[index] =\n{X[index]}")
print(f"Y[index] = {Y[index]}")

X[index] =
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Y[index] = [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


As we can see: Our dataset is more or less correct now, except that `X` build quite slow. A threaded, or multi-process version of it will be desirable.

### <s>Shuffling</s>
We'd better shuffle `X` and `Y` (together).

In [17]:
A = np.arange(7*2).reshape((7,2))
A

array([[ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9],
       [10, 11],
       [12, 13]])

In [18]:
np.random.shuffle(A)
A

array([[12, 13],
       [ 6,  7],
       [10, 11],
       [ 0,  1],
       [ 4,  5],
       [ 2,  3],
       [ 8,  9]])

Well, wait a minute, actually `train_test_split` below already incorporates shuffling mechanism. Let's not make additional shuffling here.

### Train/Validation/Test Split

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
train_test_split?

In [21]:
X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size=0.2)
X_train_val.shape, X_test.shape

((7891272, 10, 10), (1972818, 10, 10))

In [22]:
(X_train_val.shape[0] + X_test.shape[0]) - n_instances

0

In [23]:
Y.shape

(9864090, 10)

## Model

In [24]:
np.product(X.shape[1:])

100

In [25]:
#https://keras.io/api/layers/reshaping_layers/reshape/
#https://keras.io/api/layers/activation_layers/softmax/
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=X.shape[1:]),
    #keras.layers.Dense(np.product(X.shape[1:]), activation="relu"),
    keras.layers.Dense(2*np.product(X.shape[1:]), activation="relu"),
    keras.layers.Dense(np.product(X.shape[1:])),
    keras.layers.Reshape(X.shape[1:]),
    keras.layers.Softmax(axis=-1),
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 200)               20200     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
reshape (Reshape)            (None, 10, 10)            0         
_________________________________________________________________
softmax (Softmax)            (None, 10, 10)            0         
Total params: 40,300
Trainable params: 40,300
Non-trainable params: 0
_________________________________________________________________


In [27]:
# add some callbacks before beginning training.
checkpoint_cb = keras.callbacks.ModelCheckpoint("vanilla_NN_model.h5")

model.fit(X_train_val,
         Y_train_val,
         #steps_per_epoch=60_000,
         epochs=3,
         validation_split=0.2,
         verbose=True,
         callbacks=[checkpoint_cb],
)

Epoch 1/3
197282/197282 [==============================] - 373s 2ms/step - loss: 0.0297 - val_loss: 3.4396e-04
Epoch 2/3
197282/197282 [==============================] - 363s 2ms/step - loss: 2.4963e-04 - val_loss: 1.9813e-04
Epoch 3/3
197282/197282 [==============================] - 359s 2ms/step - loss: 1.3780e-04 - val_loss: 1.3065e-04


In [28]:
model.evaluate(X_test, Y_test)

61651/61651 [==============================] - 87s 1ms/step - loss: 1.2123e-04


0.0001212266506627202

In [30]:
(1, *X.shape[1:])

(1, 10, 10)

In [29]:
array = [0, 9, 7, 2]
one_hot(array)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

In [31]:
input_array = np.zeros((1, *X.shape[1:]), dtype=np.float32)
array = [0, 9, 7, 2]
input_array[0, :len(array), :] = one_hot(array)
input_array

array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

In [32]:
input_array.shape

(1, 10, 10)

In [33]:
y_pred = model.predict(input_array)
y_pred

array([[[1.00000000e+00, 1.25839168e-32, 3.87253975e-26, 3.14080867e-14,
         3.80954247e-30, 7.10799203e-30, 9.56593145e-25, 7.84007830e-32,
         3.26731961e-33, 0.00000000e+00],
        [1.64768271e-14, 2.31965217e-07, 7.14659146e-15, 9.99994993e-01,
         4.71040858e-06, 5.90644129e-19, 4.78312472e-20, 1.64960406e-10,
         1.25448426e-13, 2.96435836e-31],
        [5.37284990e-08, 6.20236911e-04, 9.99180496e-01, 1.99015878e-04,
         1.20307050e-07, 1.10409848e-09, 5.87335666e-11, 1.56619474e-11,
         2.31073743e-15, 4.79669993e-16],
        [6.45592927e-14, 9.97052312e-01, 2.94772349e-03, 5.21008098e-11,
         2.55477497e-11, 2.37600467e-14, 1.03858276e-16, 3.04541167e-16,
         6.01100777e-20, 1.02113676e-15],
        [6.92680380e-24, 8.79716885e-04, 1.48970070e-09, 2.28879119e-21,
         9.99120295e-01, 5.77007379e-26, 4.24751751e-25, 6.44041290e-21,
         6.41836234e-14, 1.79858847e-24],
        [0.00000000e+00, 2.62699312e-29, 1.93556767e-22, 1.2

In [84]:
y_pred.shape, Y_test.shape

((1, 10, 10), (1972818, 10))

In [35]:
indices = np.argmax(y_pred, axis=-1)
indices

array([[0, 3, 2, 1, 4, 5, 6, 7, 8, 9]])

In [37]:
np.array(array)[indices[0][:len(array)]]

array([0, 2, 7, 9])

## Evaluation on `X_test`
We certainly would like to have performance measures like accuracy, precision/recall, etc. But we must first write some convenience functions to facilitate the operations.

In [38]:
X_test[0]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [39]:
np.argmax(X_test[0], axis=-1)

array([3, 6, 7, 0, 5, 8, 2, 4, 9, 1])

In [46]:
np.sum([[0, 0, 0.2], [0, 0.1, 0.9]], axis=0)

array([0. , 0.1, 1.1])

In [47]:
np.sum([[0, 0, 0.2], [0, 0.1, 0.9]], axis=-1)

array([0.2, 1. ])

As we can see, `X_test` in this form is inconvenient for us to calculate the accuracy, or even to just to convert it back to the corresponding array.

In [40]:
X_test.shape[0]

1972818

In [41]:
Y_test[0]

array([3., 9., 6., 0., 7., 4., 1., 2., 5., 8.], dtype=float32)

In [43]:
np.argmax(X_test[0], axis=-1)[Y_test[0].astype(int)]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [73]:
class Sorter:
    def __init__(self, model):
        self.model = model

    def prettier(self, x, y):
        """
        x.shape = (10,10)
        """
        somme = np.sum(x, axis=-1)
        first_zero_index = -1
        for i, s in enumerate(somme):
            if s > 10**(-6):
                first_zero_index = i
        if first_zero_index == -1:
            length = 10
        else:
            length = first_zero_index + 1
        xx = np.argmax(x[:length], axis=-1)
        sort_indices = y.astype(int)[:length]
        yy = xx[sort_indices]
        return xx, yy

In [81]:
i = np.random.choice(range(X_test.shape[0]))
print(f"i = {i}")
x, y = X_test[i], Y_test[i]
sorter = Sorter(model)
sorter.prettier(x, y)

i = 1011350


(array([1, 7, 0, 5, 2, 9, 6, 8]), array([0, 1, 2, 5, 6, 7, 8, 9]))

In [85]:
class Sorter:
    def __init__(self, model):
        self.model = model

    def lenlen(self, x):
        somme = np.sum(x, axis=-1)
        first_zero_index = -1
        for i, s in enumerate(somme):
            if s > 10**(-6):
                first_zero_index = i
        if first_zero_index == -1:
            length = 10
        else:
            length = first_zero_index + 1
        return length

    def prettier(self, x, y):
        """
        x.shape = (10,10)
        """
        length = self.lenlen(x)
        xx = np.argmax(x[:length], axis=-1)
        sort_indices = y.astype(int)[:length]
        yy = xx[sort_indices]
        return xx, yy
    
    def evaluate(self, X, Y):
        Y_pred = self.model.predict(X)  # of shape (n_instances, 10, 10)
        Y = Y.astype(int)               # of shape (n_instances, 10)
        m = X.shape[0]
        n_correct = 0
        for i, x in enumerate(X):
            length = self.lenlen(x)
            y_pred = Y_pred[i]
            y_pred_sparse = np.argmax(y_pred, axis=-1)
            n_correct += np.array_equal(Y[i], y_pred_sparse)
        print(f"acc = {n_correct/m}")


In [86]:
sorter = Sorter(model)

In [ ]:
sorter.evaluate(X_test, Y_test)